In [ ]:
from transformers import pipeline

In [ ]:
model_id = "openai/whisper-small"
pipe = pipeline("automatic-speech-recognition", model=model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
!pip install beautifulsoup4 requests

import requests
from bs4 import BeautifulSoup

In [ ]:
import json
import re

In [ ]:
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        generate_kwargs={
            "task": "transcribe"
        },
        chunk_length_s=30,
        batch_size=8,
    )
    return output["text"]

def zero_shot_classification(text):

    transcribed_text = text

    url = 'https://www.producthunt.com/categories'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    h2_tags = soup.find_all('h2')
    h2_texts = [tag.text.strip() for tag in h2_tags]
    categories = h2_texts

    results = classifier(transcribed_text, candidate_labels=categories)
    label_score_pairs = sorted(zip(results['labels'], results['scores']), key=lambda x: x[1], reverse=True)[:3]
    data = {label: score for label, score in label_score_pairs}
    return data

def get_products(all_products):
    input_dict = json.loads(all_products.replace("'", '"'))
    formatted_category = re.sub(r'[ &]+', '-', list(input_dict.keys())[0]).lower()
    full_url = f"https://www.producthunt.com/categories/{formatted_category}"
    response = requests.get(full_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = [div.get_text().strip() for div in soup.select('div[class="text-18 font-semibold text-blue"]')]
    descriptions = [div.get_text().strip() for div in soup.select('div[class="text-14 sm:text-16 font-normal text-light-gray mb-6"]')]
    products = list(zip(titles, descriptions))[:5]

    html_content = ""
    for title, description in products:
            html_content += f'''
            <div style="border: 1px solid #ccc; border-radius: 10px; padding: 20px; margin-bottom: 10px;
                        background: linear-gradient(to right, #f9f9f9, #e8e8e8); color: #333;">
                <h2 style="text-decoration: none; color: #2c3e50;">{title}</h2>
                <p style="color: #34495e;">{description}</p>
            </div>
            '''
    return html_content

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
import gradio as gr

In [ ]:
css = """
.container {
    height: 100%;
}
.scrollable-content {
        flex-grow: 1;
        overflow-y: auto; /* Make content within scrollable */
        padding: 10px;
}
"""

In [ ]:
with gr.Blocks(css=css) as demo:
    with gr.Row(elem_classes=["container"]):
        with gr.Column():
            audio_input = gr.Audio(sources="upload", type="filepath", label="Your audio file")
            process_button = gr.Button("Process")
            transcription_output = gr.TextArea(label="Transcribed text", lines=5, placeholder="Transcribed text will appear here...", interactive=True)
            classify_button = gr.Button("Classify Text")
            classification_output = gr.Textbox(label="Classification Results", placeholder="Classification results will appear here...")
            get_product_button = gr.Button("Get opportunities")

            process_button.click(transcribe_speech, inputs=audio_input, outputs=transcription_output)
            classify_button.click(zero_shot_classification, inputs=transcription_output, outputs=classification_output)

        with gr.Column(elem_classes=["scrollable-content"]):
            get_product_output = gr.HTML()
            get_product_button.click(get_products, inputs=classification_output, outputs=get_product_output)

    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7dfbb0897a75144873.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
